#### Import Libraries

In [1]:
# %% loading EASYMORE
from easymore.easymore import easymore
from pathlib import Path
from shutil import copyfile
from datetime import datetime
import geopandas as gpd
import time

In [2]:
# %%Control file handling
controlFolder = Path('../0_control_files')  # Easy access to control file folder
controlFile = 'control_active.txt'          # Store the name of the 'active' file in a variable

In [3]:
#Function to extract a given setting from the control file
def read_from_control( file, setting ):
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            # ... find the line with the requested setting
            if setting in line and not line.startswith('#'):
                break    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
    # Return this value   
    return substring

In [4]:
#Function to specify a default path
def make_default_path(suffix):
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    defaultPath = rootPath / suffix 
    return defaultPath

In [5]:
# %% Get the domain folder
domain_name = read_from_control(controlFolder/controlFile,'domain_name')
domainFolder = 'domain_' + domain_name

In [6]:
#  %% Get the target shapefile
target_shp_path = read_from_control(controlFolder/controlFile,'river_basin_shp_path')
# Specify default path if needed
if target_shp_path == 'default':
    target_shp_path = make_default_path('domain_'+domain_name+'/shapefiles/catchment/') # outputs a Path()
else:
    target_shp_path = Path(target_shp_path) # make sure a user-specified path is a Path()
target_shp_name = read_from_control(controlFolder/controlFile,'river_basin_shp_name')
target_shp      = target_shp_path / target_shp_name
target_shp_ID   = read_from_control(controlFolder/controlFile,'river_basin_shp_rm_hruid')


In [7]:
target_shp = str(target_shp).replace('\\mnt\\c\\','C:\\')

In [8]:
target = gpd.read_file(target_shp)
target.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
# %% Get the source forcing file
source_nc_path  = read_from_control(controlFolder/controlFile,'source_nc_path')
# Specify default path if needed
if source_nc_path == 'default':
    source_nc_path = make_default_path('domain_{}/forcing/'.format(domain_name)) # outputs a Path()
else:
    source_nc_path = Path(source_nc_path) # make sure a user-specified path is a Path()
source_nc_name  = read_from_control(controlFolder/controlFile,'source_nc_name')
source_nc     = str(source_nc_path / source_nc_name)

In [10]:
outdir = read_from_control(controlFolder/controlFile,'remapping_out')
if outdir == 'default':
    outdir = make_default_path('domain_{}/forcing/'.format(domain_name)) # outputs a Path()
else:
    outdir = Path(outdir) # make sure a user-specified path is a Path()
outdir = str(outdir).replace('\\mnt\\c\\','C:\\')+'/'

In [11]:
# %% Get the list of variable neames
var_names       = read_from_control(controlFolder/controlFile,'var_names').split(', ')

In [12]:
original_shp = gpd.read_file(str(target_shp).replace("\\","/"))
original_shp = original_shp.set_crs('epsg:4326')
original_shp = original_shp.to_crs('epsg:4326')
original_shp.set_geometry('geometry')
original_shp.to_file(target_shp)

c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [13]:
target_shp = str(target_shp).replace('\\mnt\\c\\','C:\\')
target = gpd.read_file(target_shp)
target.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
# %% initializing EASYMORE object
esmr = easymore()
# specifying EASYMORE objects
# name of the case; the temporary, remapping and remapped file names include case name
esmr.case_name                = 'RDRS_{}'.format(domain_name)
# temporary path that the EASYMORE generated GIS files and remapped file will be saved
esmr.temp_dir                 = 'temporary{}/'.format(domain_name)
# name of target shapefile that the source netcdf files should be remapped to
esmr.target_shp               = target_shp
                                #'../shape_file/cat_pfaf_81_MERIT_Hydro_v07_Basins_v01_bugfix1_WGS84.shp'
esmr.target_shp_ID            = target_shp_ID
# name of netCDF file(s); multiple files can be specified with *
esmr.source_nc                = str(source_nc).replace('\\mnt\\c\\','C:\\') 
                                #'/project/6008034/baha2501/NA/forcing/NA_RDRS_v2.1_1980010112.nc'
                                
# name of variables from source netCDF file(s) to be remapped
esmr.var_names                = var_names

# rename the variables from source netCDF file(s) in the remapped files;
# it will be the same as source if not provided
#esmr.var_names_remapped       = ['RDRS']
# name of variable longitude in source netCDF files
esmr.var_lon                  =  read_from_control(controlFolder/controlFile,'var_lon')
# name of variable latitude in source netCDF files
esmr.var_lat                  = read_from_control(controlFolder/controlFile,'var_lat')
# name of variable time in source netCDF file; should be always time
esmr.var_time                 = read_from_control(controlFolder/controlFile,'var_time')
# location where the remapped netCDF file will be saved
#esmr.output_dir               = '/mnt/c/Users/5600x/Desktop/GWF/MESH_CWARHM/MESH-Data/domain_BowAtBanff/forcing/'
esmr.output_dir               = outdir
# format of the variables to be saved in remapped files,
# if one format provided it will be expanded to other variables
esmr.format_list              = read_from_control(controlFolder/controlFile,'format_list').split(', ')
# fill values of the variables to be saved in remapped files,
# if one value provided it will be expanded to other variables
esmr.fill_value_list          = read_from_control(controlFolder/controlFile,'fill_value_list').split(', ')
# if required that the remapped values to be saved as csv as well
#esmr.save_csv                 = True
#esmr.complevel                 =  9
# if uncommented EASYMORE will use this and skip GIS tasks
#esmr.remap_csv                = 'temporary78/subbasin_select/RDRS_78_remapping.csv' # RDRS_81_remapping.csv

EASYMORE version 0.0.4 is initiated.


In [15]:
# # create source shapefile 
esmr.NetCDF_SHP_lat_lon()
# create the source shapefile for case 1 and 2 if shapefile is not provided
if (esmr.case == 1 or esmr.case == 2)  and (esmr.source_shp == ''):
    if esmr.case == 1:
        if hasattr(esmr, 'lat_expanded') and hasattr(esmr, 'lon_expanded'):
            esmr.lat_lon_SHP(esmr.lat_expanded, esmr.lon_expanded,\
                esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')
        else:
            esmr.lat_lon_SHP(esmr.lat, esmr.lon,\
                esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')
    else:
        esmr.lat_lon_SHP(esmr.lat, esmr.lon,\
            esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')
    print('EASYMORE is creating the shapefile from the netCDF file and saving it here:')
    print(esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')

EASYMORE detects case 2 - rotated lat/lon
EASYMORE is creating the shapefile from the netCDF file and saving it here:
temporaryBowAtBanff/RDRS_BowAtBanff_source_shapefile.shp


In [16]:
src = gpd.read_file(esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')

In [ ]:
src = src.set_crs('EPSG:4326')

In [ ]:
src.crs

In [17]:
src = src [src['lon_s']>-179]
src.to_file(esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')

c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [18]:
# add the source shapefile 
esmr.source_shp                =   esmr.temp_dir+esmr.case_name+'_source_shapefile.shp'
esmr.source_shp_lat            =  'lat_s' # name of column latitude in the source shapefile
esmr.source_shp_lon            =  'lon_s' # name of column longitude in the source shapefile
esmr.source_shp_ID             =  'ID_s' # name of column ID in the source shapefile

In [20]:
esmr.nc_remapper()


remap file is provided; EASYMORE will use this file and skip calculation of remapping
EASYMORE will remap variable  RDRS_v2.1_A_PR0_SFC  from source file to variable  RDRS_v2.1_A_PR0_SFC  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_FI_SFC  from source file to variable  RDRS_v2.1_P_FI_SFC  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_FB_SFC  from source file to variable  RDRS_v2.1_P_FB_SFC  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_TT_09944  from source file to variable  RDRS_v2.1_P_TT_09944  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_UVC_09944  from source file to variable  RDRS_v2.1_P_UVC_09944  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_P0_SFC  from source file to variable  RDRS_v2.1_P_P0_SFC  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_HU_09944  from source file to variable  RDRS_v2.1_P_HU_09944  in remapped NeCDF file
EASYMORE case exists in the remap fil

In [21]:
target.crs

,COMID,unitarea,geometry
0,71032440,107.012467,"POLYGON ((-116.47042 51.73542, -116.46708 51.7..."
1,71032422,50.929339,"POLYGON ((-116.34125 51.71542, -116.34042 51.7..."
2,71032409,65.791372,"MULTIPOLYGON (((-116.11542 51.68625, -116.1145..."
3,71032292,25.591308,"POLYGON ((-116.07792 51.65458, -116.07458 51.6..."
4,71032217,56.835431,"POLYGON ((-116.23708 51.65208, -116.23458 51.6..."
5,71032185,71.512958,"POLYGON ((-116.44875 51.63458, -116.44708 51.6..."
6,71032102,48.068860,"POLYGON ((-116.08792 51.59125, -116.08542 51.5..."
7,71031893,70.709054,"POLYGON ((-116.38458 51.56375, -116.38292 51.5..."
8,71031838,65.100840,"POLYGON ((-116.03125 51.50458, -116.03042 51.5..."
9,71031687,27.933849,"MULTIPOLYGON (((-116.03042 51.43875, -116.0270..."
